# Energy Prediction Exercise

Please reference the `READ.ME` before getting started

## Goal

Create a model that can accurately (+/- 10%) predict the energy consumption on December 31st using nothing but energy data and publicly accessible weather data.

## Data Import

Import the data and convert the csv data into a Pandas dataframe. Then plot using matplotlib

In [ ]:
import pandas as pd
raw_df = pd.read_csv('./data/cleaned_data.csv')
raw_df

## Graphical View


In [ ]:
raw_df.reset_index().plot.scatter(x = 'index',  y='consumption')

In [ ]:
raw_df.tail(24).reset_index().plot.scatter(x = 'index',  y='consumption')

## Minor Transformations and Cleanup

1. Create a new `timestamp` column that transforms the stringified timestamp to python datetime objects.
2. Set the new column to the dataframes index column.
3. Drop the old `Timestamp` column as it is no longer needed.
4. Add a new `hour` and `dow` (day of week) column for additional timeframe analysis using built in pandas methods
5. Duplicate the dataframe for a training set
6. Remove the last day's data (we don't want to train the models with the data we're trying to predict!)

In [ ]:

raw_df['timestamp'] = pd.to_datetime(raw_df['Timestamp'])
raw_df.drop(columns=['Timestamp'], inplace=True)
raw_df.set_index('timestamp', inplace=True)

raw_df['hour'] = raw_df.index.hour
raw_df['dow'] = raw_df.index.dayofweek

df = raw_df.copy()
df.drop(df.tail(24).index,inplace=True)
df

## Graphical Comparison

Plot Hour, DoW, Outdoor Air Temp, CDD, HDD vs Energy

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,10)
figure, axis = plt.subplots(3, 2)
axis[0, 0].scatter(df.index, df['consumption'])
axis[0, 0].set_title('Time vs Consumption')
axis[0, 1].scatter(df['hour'], df['consumption'])
axis[0, 1].set_title('Hour vs Consumption')
axis[1, 0].scatter(df['dow'], df['consumption'])
axis[1, 0].set_title('Day of Week vs Consumption')
axis[1, 1].scatter(df['cdd'], df['consumption'])
axis[1, 1].set_title('CDD vs Consumption')
axis[2, 0].scatter(df['hdd'], df['consumption'])
axis[2, 0].set_title('HDD vs Consumption')
axis[2, 1].scatter(df['avg_tmp'], df['consumption'])
axis[2, 1].set_title('Avg Temp vs Consumption')
plt.show()

## Let's Start Modeling!!

We're going to start with a simple regression model which is essentially finding the line of best fit for the data.

Our first variable we're going to try and fit is Hour vs Consumption using the Stat Models library.

After initial testing, we are going to use a 5th degree polynomial which we can create with Scikit-learn.

Our steps include:
1. Make a fresh copy of the dataframe that we can manipulate
3. Apply our 5th degree polynomial by creating 5 new columns in the dataframe and applying the scikit-learn PolynomialFeatures class
4. Separate out our data into independent (feature in ml lingo) and dependent (label in ml lingo) variables

Let's check out what the independent data looks like!


In [ ]:
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures


hour_df = df.copy()

hour_poly_degree = 5
hour_df[[f'hour:poly={i}' for i in range(0, hour_poly_degree + 1)]] = PolynomialFeatures(hour_poly_degree).fit_transform(df[['hour']])

hour_x = hour_df.drop(columns=['hour', 'dow', 'cdd', 'hdd', 'avg_tmp', 'consumption'])
y = df['consumption']

hour_x

5. Then we fit our data using statsmodels Linear Regression Ordinary Least Squares model

Next lets checkout the summary statistics!

In [ ]:
hour_model = sm.OLS(y, hour_x).fit()
hour_model.summary()

## Analyze the Results

We are interested in the R-Squared and P>|t| values above

The R-Squared is very important. Utilties and ESCO companies follow IPMVP which is the International Performance Measurement and Verification Protocol to verify energy savings. You can learn more about IPMVP [HERE](https://evo-world.org/en/products-services-mainmenu-en/protocols/ipmvp) however, the important thing to know is we need a r-squared value greater than 0.7 to be deemed statistically significant.

The p-value is also very important when looking at the summary statistics. This value can tell us weather the independent variable had a statistically significant impact on the model. Typically any value less than 0.05 is considered statistically significant. If the value is greater than 0.05 you typically drop that variable and refit the model without the bad variable.

## Let's see how this model looks!

Even though the R-Squared value is below 0.7, lets still take a look and see how this model did!

In [ ]:
# Show Hourly Predicted Model
hours = hour_x.copy().iloc[:24,:]
hours['Predicted Consumption'] = hour_model.predict(hours)
hours['hour'] = hours.index.hour

plt.scatter(df['hour'], df['consumption'])
plt.title('Hour vs Consumption')
plt.plot(hours['hour'], hours['Predicted Consumption'], color='red')
plt.show()

year_df = hour_x.copy()
year_df['Predicted Consumption'] = hour_model.predict(year_df)
year_df['hour'] = year_df.index.hour
plt.scatter(df.index, df['consumption'])
plt.plot(year_df.index, year_df['Predicted Consumption'], color='red')
plt.show()


## How to do the other variables look?

So I ran the numbers on each of the variables and the results are as follows:

- DoW is best with a 5th degree polynomial applied
- CDD and HDD work best with no polynomial applied (they are suppose to be linear)
- Avg Temp works best with a 6th degree polynomial applied.

In [ ]:
## DoW Model
dow_poly_degree = 5
dow_df = df.copy()
dow_df[[f'hour:poly={i}' for i in range(0, dow_poly_degree + 1)]] = PolynomialFeatures(dow_poly_degree).fit_transform(df[['dow']])
dow_x = dow_df.drop(columns=['hour', 'dow', 'cdd', 'hdd', 'avg_tmp', 'consumption'])
dow_x = sm.add_constant(dow_x)
y = df['consumption']
dow_model = sm.OLS(y, dow_x).fit()

# CDD Model
cdd_df = df.copy()
cdd_df = cdd_df[(cdd_df.cdd != 0)]
cdd_x = cdd_df.drop(columns=['hour', 'dow', 'hdd', 'avg_tmp', 'consumption'])
cdd_x = sm.add_constant(cdd_x)
y = cdd_df['consumption']
cdd_model = sm.OLS(y, cdd_x).fit()

# HDD Model
hdd_df = df.copy()
hdd_df = hdd_df[(hdd_df.hdd != 0)]
hdd_x = hdd_df.drop(columns=['hour', 'dow', 'cdd', 'avg_tmp', 'consumption'])
hdd_x = sm.add_constant(hdd_x)
y = hdd_df['consumption']
hdd_model = sm.OLS(y, hdd_x).fit()

# Avg Temp Model
avg_tmp_poly_degree = 6
avg_tmp_df = df.copy()
avg_tmp_df[[f'hour:poly={i}' for i in range(0, avg_tmp_poly_degree + 1)]] = PolynomialFeatures(avg_tmp_poly_degree).fit_transform(df[['avg_tmp']])
avg_tmp_x = avg_tmp_df.drop(columns=['hour', 'dow', 'cdd', 'hdd', 'avg_tmp', 'consumption'])
avg_tmp_x = sm.add_constant(avg_tmp_x)
y = df['consumption']
avg_temp_model = sm.OLS(y, avg_tmp_x).fit()

# Predict Individual Models

# Predict DoW Predicted Model
dow = dow_x.copy().iloc[:(10 * 24),:].iloc[::24, :].iloc[3:]
dow['Predicted Consumption'] = dow_model.predict(dow)
dow['dow'] = dow.index.dayofweek

# Predict CDD Model
cdd = cdd_x.copy()
cdd['Predicted Consumption'] = cdd_model.predict(cdd)
cdd['cdd'] = df['cdd']

# Show HDD Model
hdd = hdd_x.copy()
hdd['Predicted Consumption'] = hdd_model.predict(hdd)
hdd['hdd'] = df['hdd']

# Show Avg Temp Model
temp = avg_tmp_x.copy()
temp['Predicted Consumption'] = avg_temp_model.predict(temp)
temp['avg_tmp'] = df['avg_tmp']

# Show all models on graphs
figure, axis = plt.subplots(2, 2)
axis[0, 0].scatter(df['dow'], df['consumption'])
axis[0, 0].plot(dow['dow'], dow['Predicted Consumption'], color='red')
axis[0, 0].set_title('Day of Week vs Consumption')
axis[0, 1].scatter(df['cdd'], df['consumption'])
axis[0, 1].plot(cdd['cdd'], cdd['Predicted Consumption'], color='red')
axis[0, 1].set_title('CDD vs Consumption')
axis[1, 0].scatter(df['hdd'], df['consumption'])
axis[1, 0].plot(hdd['hdd'], hdd['Predicted Consumption'], color='red')
axis[1, 0].set_title('HDD vs Consumption')
axis[1, 1].scatter(df['avg_tmp'], df['consumption'])
axis[1, 1].set_title('Avg Temp vs Consumption')
axis[1, 1].plot(temp['avg_tmp'], temp['Predicted Consumption'], color='red')



## Let's Improve it!

Next let's try adding in some more variables! Let's add CDD and HDD to the hourly variable and see if we can make it any better.

Let's take a peak at the data after we add in the extra independent variables

In [ ]:
hour_poly_degree = 5
new_df = df.copy()
new_df[[f'hour:poly={i}' for i in range(0, hour_poly_degree + 1)]] = PolynomialFeatures(hour_poly_degree).fit_transform(new_df[['hour']])

hour_cdd_hdd_x = new_df.drop(columns=['dow', 'hour', 'avg_tmp', 'consumption'])
y = df['consumption']

hour_cdd_hdd_x

Let's finish it off and see the model!

Note that get a more readable graph, the consumption values are averaged by day.

In [ ]:
hour_cdd_hdd_model = sm.OLS(y, hour_cdd_hdd_x).fit()

temp_df = df.copy()
temp_df['Predicted Consumption'] = hour_cdd_hdd_model.predict(hour_cdd_hdd_x)
ndf = temp_df.resample('D').mean()
plt.scatter(ndf.index, ndf['consumption'])
plt.title('Consumption vs Time')
plt.xlabel('Time (daily avg)')
plt.ylabel('Consumption (kWh)')
plt.plot(ndf.index, ndf['Predicted Consumption'], color='red')

hour_cdd_hdd_model.summary()

## Does this model meet our Goal?

Remember our goal was to be within +/- 10% of the actual consumption value for December 31st

In [ ]:

predicted_df = raw_df.copy()
predicted_df[[f'hour:poly={i}' for i in range(0, 5 + 1)]] = PolynomialFeatures(5).fit_transform(predicted_df[['hour']])
final_model_x = predicted_df.drop(columns=['dow', 'hour', 'avg_tmp', 'consumption'])

predicted_df['Predicted Consumption'] = hour_cdd_hdd_model.predict(final_model_x)
predicted_interval = predicted_df.iloc[-24:]

plt.scatter(predicted_interval.index, predicted_interval['consumption'])
plt.title('Consumption vs Time')
plt.xlabel('Time')
plt.ylabel('Consumption (kWh)')
plt.plot(predicted_interval.index, predicted_interval['Predicted Consumption'], color='red')
plt.show()


Let's checkout the numbers (fingers crossed)

In [ ]:
from IPython.display import display

consumption_df = predicted_interval[['consumption', 'Predicted Consumption']]
actual = consumption_df['consumption'].sum()
predicted = consumption_df['Predicted Consumption'].sum()
percent_diff = ((abs(actual - predicted)) / ((actual + predicted) / 2)) * 100

display(consumption_df)
display('Totals')
display(consumption_df.sum())
display(f"Difference: {actual-predicted}")
display(f"% Difference: {round(percent_diff, 2)}%")

